# Exercise 11: Entity and Relation Extraction

## Task 1: Relation extraction from Wikipedia articles

Use Wikipedia to extract the relation `directedBy(Movie, Person)` by applying pattern based heuristics that utilize: *Part Of Speech Tagging*, *Named Entity Recognition* and *Regular Expressions*.

#### Required Library: SpaCy
- ```conda install -y spacy```
- ```python -m spacy download en```

In [1]:
import urllib.request, json, csv, re
import spacy
nlp = spacy.load('en')

In [2]:
#read tsv with input movies
def read_tsv():
    movies=[]
    with open('movies.tsv','r') as file:
        tsv = csv.reader(file, delimiter='\t')
        next(tsv) #remove header
        movies = [{'movie':line[0], 'director':line[1]} for line in tsv]
    return movies

#parse wikipedia page
def parse_wikipedia(movie):
    txt = ''
    try:
        with urllib.request.urlopen('https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro=&explaintext=&titles='+movie) as url:
            data = json.loads(url.read().decode())
            txt = next (iter (data['query']['pages'].values()))['extract']
    except:
        pass
    return txt

#### 1) Parse the raw text of a Wikipedia movie page and extract named (PER) entities.

In [37]:
def find_PER_entities(txt):
    split = nlp(txt)
    persons = [ent.text for ent in split.ents if ent.label_ == 'PERSON']
    return persons

#### 2) Given the raw text of a Wikipedia movie page and the extracted PER entities, find the director.

In [38]:
import re
def find_matches(file_text, regexp_string):
    #Compile a regular expression 
    regexp = re.compile(regexp_string)
    
    #Find all matches with the given regular expression
    matches = re.findall(regexp, file_text)
    
    return matches

In [63]:
def find_director(txt, persons):
    director = ''
    if len(persons)>0:
        for i in range(0,len(persons)):
            temp = find_matches(txt, "[a-z]+ directed by " + persons[i])
            if temp!=[]:
                director = persons[i]
    
    return director

In [70]:
movies = read_tsv()

statements=[]
for m in movies:

        txt = parse_wikipedia(m['movie'])
        persons = find_PER_entities(txt)
        try:
            director = find_director(txt, persons)
            if director != '':
                statements.append(m['movie'] + ' is directed by ' + director + '.')
        except:
            continue

#### 3) Compute the precision and recall based on the given ground truth (column Director from tsv file) and show examples of statements that are extracted.

In [97]:
# compute precision and recall
missing = len(movies) - len(statements)
matches = 0
mistakes = 0
for i in range(0,len(statements)):
    dp = statements[i].split(" ")
    movie = dp[0]
    split_ind = dp.index('by')
    director = dp[split_ind+1 :]
    name = (' '.join(director)).split('.')[0]
    for i in range(0,len(movies)):
        if movies[i]['movie'] == movie:
            if movies[i]['director'] == name:
                matches = matches + 1
            else:
                mistakes = mistakes + 1
                

precision = matches / (matches + mistakes)
recall =  matches / (matches + missing)
print ('Precision:',precision)
print ('Recall:',recall)


print('***Sample Statements***')
for s in statements[:5]:
    print (s)

Precision: 0.8262910798122066
Recall: 0.704
***Sample Statements***
13_Assassins_(2010_film) is directed by Takashi Miike.
14_Blades is directed by Daniel Lee.
22_Bullets is directed by Richard Berry.
The_A-Team_(film) is directed by Joe Carnahan.
Alien_vs_Ninja is directed by Seiji Chiba.


## Task 2: Named Entity Recognition using Hidden Markov Model


Define a Hidden Markov Model (HMM) that recognizes Person (*PER*) entities.
Particularly, your model must be able to recognize pairs of the form (*firstname lastname*) as *PER* entities.
Using the given sentences as training and test set:

In [98]:
training_set=['The best blues singer was Bobby Bland while Ray Charles pioneered soul music .', \
              'Bobby Bland was just a singer whereas Ray Charles was a pianist , songwriter and singer .' \
              'None of them lived in Chicago .']

test_set=['Ray Charles was born in 1930 .', \
          'Bobby Bland was born the same year as Ray Charles .', \
          'Muddy Waters is the father of Chicago Blues .']

#### 1) Annotate your training set with the labels I (for PER entities) and O (for non PER entities).
	
    *Hint*: Represent the sentences as sequences of bigrams, and label each bigram.
	Only bigrams that contain pairs of the form (*firstname lastname*) are considered as *PER* entities.

In [111]:
#Bigram Representation
def getBigrams(sents):
    return [b[0]+' '+b[1] for l in sents for b in zip(l.split(' ')[:-1], l.split(' ')[1:])]

def getBigrams2(sentence):
    return [b[0]+' '+b[1] for b in zip(sentence.split(' ')[:-1], sentence.split(' ')[1:])]

annotations_per_sentence = []
PER = ['Bobby Bland', 'Ray Charles']
for sent in training_set:
    bigrams = getBigrams2(sent)
    annotations = []
    for b in bigrams:
        check = b in PER
        if(check):
            annotations.append('I')
        else:
            annotations.append('O')
    annotations_per_sentence.append(annotations)
    

print(annotations_per_sentence)

'''
bigrams = getBigrams(training_set)
print(bigrams)
#Annotation
PER = ['Bobby Bland', 'Ray Charles']
annotations = []
for b in bigrams:
    check = b in PER
    if(check):
        annotations.append('I')
    else:
        annotations.append('O')
print('Annotation\n', annotations,'\n')
'''

[['O', 'O', 'O', 'O', 'O', 'I', 'O', 'O', 'I', 'O', 'O', 'O', 'O'], ['I', 'O', 'O', 'O', 'O', 'O', 'O', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


"\nbigrams = getBigrams(training_set)\nprint(bigrams)\n#Annotation\nPER = ['Bobby Bland', 'Ray Charles']\nannotations = []\nfor b in bigrams:\n    check = b in PER\n    if(check):\n        annotations.append('I')\n    else:\n        annotations.append('O')\nprint('Annotation\n', annotations,'\n')\n"

#### 2) Compute the transition and emission probabilities for the HMM (use smoothing parameter $\lambda$=0.5).

    *Hint*: For the emission probabilities you can utilize the morphology of the words that constitute a bigram (e.g., you can count their uppercase first characters).

In [112]:
lambda_ = 0.5

#Transition Probabilities
transition_prob={}


#Prior
total_sentences =len(annotations_per_sentence)
Is_init = 0
Os_init = 0
for i in range(0,total_sentences):
    first_element = annotations_per_sentence[i][0]
    if first_element == 'I':
        Is_init +=1
    elif first_element == 'O':
        Os_init +=1

transition_prob['P(I|start)'] = Is_init/total_sentences
transition_prob['P(O|start)'] = Os_init/total_sentences

total = 0
O_given_O = 0
O_given_I = 0
I_given_O = 0
I_given_I = 0
for sentence in annotations_per_sentence:
    for j in range(1,len(sentence)):
        if sentence[j] == 'O' and sentence[j-1] == 'O':
            O_given_O+=1
        elif sentence[j] == 'O' and sentence[j-1] == 'I':
            O_given_I +=1
        elif sentence[j] == 'I' and sentence[j-1] == 'O':
            I_given_O += 1
        elif sentence[j] == 'I' and sentence[j-1] == 'I':
            I_given_I +=1
            
    total += len(sentence)       
        
transition_prob['P(O|O)'] = O_given_O/total
transition_prob['P(O|I)'] = O_given_I/total
transition_prob['P(I|O)'] = I_given_O/total
transition_prob['P(I|I)'] = I_given_I/total

print('Transition Probabilities\n',transition_prob, '\n')


#Emission Probabilities
emission_prob={}

        
default_emission = 0.01

emission_prob['P(2_upper|O)'] = ...
emission_prob['P(2_upper|I)'] = ...
emission_prob['P(1_upper|O)'] = ...
emission_prob['P(1_upper|I)'] = ...
emission_prob['P(0_upper|O)'] = ...
emission_prob['P(0_upper|I)'] = ...

print('Emission Probabilities\n', emission_prob, '\n')

Transition Probabilities
 {'P(I|start)': 0.5, 'P(O|start)': 0.5, 'P(O|O)': 0.7428571428571429, 'P(O|I)': 0.11428571428571428, 'P(I|O)': 0.08571428571428572, 'P(I|I)': 0.0} 



"\n#Emission Probabilities\nemission_prob={}\n\n        \ndefault_emission = ...\n\nemission_prob['P(2_upper|O)'] = ...\nemission_prob['P(2_upper|I)'] = ...\nemission_prob['P(1_upper|O)'] = ...\nemission_prob['P(1_upper|I)'] = ...\nemission_prob['P(0_upper|O)'] = ...\nemission_prob['P(0_upper|I)'] = ...\n\nprint('Emission Probabilities\n', emission_prob, '\n')\n"

#### 3) Predict the labels of the test set and compute the precision and the recall of your model.

In [10]:
#Prediction
bigrams = getBigrams(test_set)
entities=[]
prev_state='start'
for b in bigrams:
    I_prob = ...
    O_prob = ...
    
    if ...:
        prev_state = 'O'
    else:
        entities.append(b)
        prev_state = 'I'

print('Predicted Entities\n', entities, '\n')

Predicted Entities
 [] 



Precision is *...%* while recall is *...%*. 

#### 4) Comment on how you can further improve this model.

...